In [ ]:
import numpy as np
from matplotlib import rcParams
import matplotlib.pyplot as plt
from scipy.signal import convolve2d, convolve
from skimage import data, measure 
from skimage.color import rgb2gray
from ipywidgets import interact, interactive, interact_manual, fixed
from IPython.display import Latex, Math, Markdown
import pandas as pd

In [ ]:
%matplotlib notebook


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# 1D examples

df = pd.read_csv("BTC_USD_2018-11-01_2019-10-31-CoinDesk.csv", index_col=False) 

plt.figure()
plt.plot(df['Closing Price (USD)'], linewidth=3)

ra7 = np.ones(7) / 7
ra30 = np.ones(30) / 30

c7 = convolve(df['Closing Price (USD)'], ra7, mode='same')
c30 = convolve(df['Closing Price (USD)'], ra30, mode='same')

plt.plot(c7, linewidth=3)
plt.plot(c30, linewidth=3)

In [ ]:
plt.figure()
plt.plot(df['Closing Price (USD)'], linewidth=3)
c7 = convolve(np.pad(df['Closing Price (USD)'], 6, mode='edge'), ra7, mode='same')
c30 = convolve(np.pad(df['Closing Price (USD)'], 29, mode='edge'), ra30, mode='same')

plt.plot(c7[7:-7], linewidth=3)
plt.plot(c30[30:-30], linewidth=3)

In [ ]:
plt.figure()
plt.plot(np.gradient(df['Closing Price (USD)']))
plt.plot(np.gradient(c30[30:-30]))

kernel = [1, -1]
# plt.plot(convolve(df['Closing Price (USD)'], kernel, mode='same')[1:])

In [ ]:
display(Markdown('### Step1: Choose Image'))
images = [data.astronaut,
         #data.brick(),
         data.camera,
         #data.cell,
         data.checkerboard,
         data.chelsea,
         data.coffee,
         #data.grass,
         #data.gravel,
         data.hubble_deep_field,
         data.moon,
         data.retina]

fig, ax1 = plt.subplots()
def showImage(imageNum=0):
    img = images[imageNum]()
    if len(img.shape) == 3:
        # convert from color to grey scale
        img = rgb2gray(img)
    ax1.imshow(img, cmap='gray')
    
w = interactive(showImage, imageNum=(0,len(images)-1, 1))
display(w)

display(Markdown('***'))
display(Markdown('### Step 2: Choose Kernel'))

def showKernel(a11=0, a12=0, a13=0, a21=0, a22=1, a23=0, a31=0, a32=0, a33=0):
    string = r'\begin{bmatrix}'
    string += r'%i & %i & %i \\' % (a11, a12, a13)
    string += r'%i & %i & %i \\' % (a21, a22, a23)
    string += r'%i & %i & %i' % (a31, a32, a33)
    string += r'\end{bmatrix}'
    display(Math(string))
    
h = interactive(showKernel, a11=(-9,9,1), a12=(-9,9,1), a13=(-9,9,1),
                a21=(-9,9,1), a22=(-9,9,1), a23=(-9,9,1),
                a31=(-9,9,1), a32=(-9,9,1), a33=(-9,9,1))
display(h)

display(Markdown('***'))
display(Markdown('### Step 3: Convolve Image with Kernel'))

fig, ax2 = plt.subplots(1,2)
def conv(w, h):
    ax2[0].clear()
    ax2[1].clear()
    
    img = images[w.children[0].value]()
    if len(img.shape) == 3:
        # convert from color to grey scale
        img = rgb2gray(img)
    
    kernel = [[h.children[0].value, h.children[1].value, h.children[2].value],
              [h.children[3].value, h.children[4].value, h.children[5].value],
              [h.children[6].value, h.children[7].value, h.children[8].value]]
    
    obs = convolve2d(img, kernel, mode="same")
    obs -= np.min(obs)
    obs = obs / np.max(obs) * np.max(img)
    
    ax2[0].imshow(img, cmap='gray')
    ax2[0].set_xlabel('Original')
    ax2[1].imshow(obs, cmap='gray')
    ax2[1].set_xlabel('Convolved')
    
y = interact_manual(conv, w=fixed(w), h=fixed(h))
y.widget.children[0].description = 'Convolve!'
display(y)